In [ ]:
%matplotlib inline


# Preprocessing signal


In [ ]:
import vital_sqi
from vital_sqi.data.signal_io import ECG_reader, PPG_reader
import os

In [ ]:
### Read an EDF file using ECG_reader
#The function returns an Signal SQI object and a Dictionary contains the information of the file settings**

In [ ]:
from vital_sqi.dataset import load_ecg, load_ppg
# ecg_data = load_ecg()
file_name = os.path.abspath('../../tests/test_data/example.edf')
ecg_data = ECG_reader(file_name, 'edf')

In [ ]:
ecg_data.info

In [ ]:
### List all of the attributes in Signal_SQI object
#1. signals: a numpy array contains the raw amplitude values of the devices
#2. sampling_rate: the sampling rate derives from the signal data
#3. wave_type: the types of signal. Only 2 types are accepted: either 'ecg' or 'ppg'
#4. sqi_indexes: a list of sqi_indexes

In [ ]:
ecg_data.__dict__

In [ ]:
all_channels = ecg_data.signals

In [ ]:
channel_1 = all_channels.iloc[:, 1]
ecg_sample_idx = int(len(all_channels)/2)

In [ ]:
ecg_sample_complex_for_tapering = \
    channel_1[ecg_sample_idx-45:ecg_sample_idx+210]
ecg_sample_complex = channel_1[ecg_sample_idx+80:ecg_sample_idx+225]

In [ ]:
import plotly.graph_objects as go
import numpy as np

In [ ]:
### We will focus on 1 QRS-complex to examine the function

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y=ecg_sample_complex))
#fig.show()
fig

In [ ]:
file_name = os.path.abspath('../../tests/test_data/ppg_smartcare.csv')
ppg_data = PPG_reader(file_name,
                    signal_idx=['PLETH'],
                    timestamp_idx=['TIMESTAMP_MS'],
                    info_idx=['PULSE_BPM', 'SPO2_PCT', 'PERFUSION_INDEX'],
                    sampling_rate=100,
                    start_datetime='2020/12/30 10:00:00')

# ppg_data = load_ppg()

In [ ]:
ppg_data.__dict__

In [ ]:
ppg_sample_idx = int(len(ppg_data.signals)/2)

In [ ]:
ppg_sample_complex_for_tapering = \
    ppg_data.signals.iloc[:, 1][ppg_sample_idx+185:ppg_sample_idx+225]
ppg_sample_complex = ppg_data.signals.iloc[:, 1][
                     ppg_sample_idx+195:ppg_sample_idx+267]

In [ ]:
### Subsequently, we also focus on 1 PPG waveform to examine the function

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ppg_sample_complex)),
                         y=ppg_sample_complex))
#fig.show()
fig

In [ ]:
## Examples on preprocessing function
#import the preprocess package

In [ ]:
from vital_sqi import preprocess

In [ ]:
### Taper data into the zerobaseline to remove the edge effect

In [ ]:
ecg_sample_tapering_zerobaseline = preprocess.preprocess_signal.taper_signal(
                                                        ecg_sample_complex,
                                                        shift_min_to_zero=True)
ppg_sample_tapering_zerobaseline = preprocess.preprocess_signal.taper_signal(
                                                        ppg_sample_complex,
                                                        shift_min_to_zero=True)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y=ecg_sample_complex,
                         name='original_signal'))
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y=ecg_sample_tapering_zerobaseline,
                         name='tapered signal'))
#fig.show(title='ecg tapering')
fig
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ppg_sample_complex)),
                         y=ppg_sample_complex,
                         name='original_signal'))
fig.add_trace(go.Scatter(x=np.arange(len(ppg_sample_tapering_zerobaseline)),
                         y=ppg_sample_tapering_zerobaseline,
                         name='tapered signal'))
#fig.show()
fig

In [ ]:
### The tapering data will pin the first and last part at the zero pivot. The remaining will be scale according to the windows format

#The default tapering method shifts the segment by the value equal to the minimum value to the zero baseline set shift_min_to_zero=False**

In [ ]:
ecg_sample_tapering_zerobaseline = \
    preprocess.taper_signal(ecg_sample_complex,shift_min_to_zero=False)
ppg_sample_tapering_zerobaseline = \
    preprocess.taper_signal(ppg_sample_complex,shift_min_to_zero=False)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y=ecg_sample_complex,
                         name='original_signal'))
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y=ecg_sample_tapering_zerobaseline,
                         name='tapered signal'))
#fig.show(title='ECG')
fig
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ppg_sample_complex)),
                         y=ppg_sample_complex,
                         name='original_signal'))
fig.add_trace(go.Scatter(x=np.arange(len(ppg_sample_complex)),
                         y=ppg_sample_tapering_zerobaseline,
                         name='tapered signal'))
#fig.show(itle='PPG')
fig

In [ ]:
### Different windows format can be used to perform tapering process
#window is imported from the scipy package (scipy.signal.window). Default is using Tukey window**

In [ ]:
import scipy.signal.windows as wd

In [ ]:
#Initialize a hann windows and cast it as a list-type.**

In [ ]:
window_ecg = list(wd.hann(len(ecg_sample_complex)))
window_ppg = list(wd.hann(len(ppg_sample_complex)))

In [ ]:
ecg_sample_tapering_hann = preprocess.taper_signal(ecg_sample_complex,
                                           window=window_ecg,
                                           shift_min_to_zero=False)
ppg_sample_tapering_hann = preprocess.taper_signal(ppg_sample_complex,
                                           window=window_ppg,
                                           shift_min_to_zero=False)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y=ecg_sample_complex,
                         name='original signal'))
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y=ecg_sample_tapering_zerobaseline,
                         name='tukey window tapering'))
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y=ecg_sample_tapering_hann,
                         name='hann window tapering'))

fig.update_layout(
    title='ECG'
)
fig

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ppg_sample_complex)),
                         y=ppg_sample_complex,
                         name='original signal'))
fig.add_trace(go.Scatter(x=np.arange(len(ppg_sample_complex)),
                         y=ppg_sample_tapering_zerobaseline,
                         name='tukey window tapering'))
fig.add_trace(go.Scatter(x=np.arange(len(ppg_sample_complex)),
                         y=ppg_sample_tapering_hann,
                         name='hann window tapering'))
fig.update_layout(
    title='PPG'
)

In [ ]:
### Example of smoothing function.
#Apply a convolutional window to smooth the signal (the default windows is flat and can be assigned with different distribution)**

In [ ]:
ecg_sample_smoothing_5 = preprocess.smooth_signal(ecg_sample_complex)
ecg_sample_smoothing_9 = preprocess.smooth_signal(ecg_sample_complex,
                                                  window_len=9)

ppg_sample_smoothing_5 = preprocess.smooth_signal(ppg_sample_complex)
ppg_sample_smoothing_9 = preprocess.smooth_signal(ppg_sample_complex,
                                                  window_len=9)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y=ecg_sample_complex,
                         name='original_signal'))
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y=ecg_sample_smoothing_5,
                         name='smoothing - sliding window length = 5'))
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y=ecg_sample_smoothing_9,
                         name='smoothing - sliding window length = 9'))
#fig.show()
fig

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ppg_sample_complex)),
                         y=ppg_sample_complex,
                         name='original_signal'))
fig.add_trace(go.Scatter(x=np.arange(len(ppg_sample_complex)),
                         y=ppg_sample_smoothing_5,
                         name='smoothing - sliding window length = 5'))
fig.add_trace(go.Scatter(x=np.arange(len(ppg_sample_complex)),
                         y=ppg_sample_smoothing_9,
                         name='smoothing - sliding window length = 9'))
#fig.show()
fig

In [ ]:
### Example of squeezing function
#We will use the default resampling function from scipy package**

In [ ]:
from scipy import signal

In [ ]:
ecg_sample_squeezing = signal.resample(ecg_sample_complex,
                                       int(len(ecg_sample_complex)/2))
ppg_sample_squeezing = signal.resample(ppg_sample_complex,
                                       int(len(ppg_sample_complex)/2))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y= ecg_sample_complex))
#fig.show(title='original data')
fig

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_squeezing)),
                         y= ecg_sample_squeezing))
#fig.show(title='squeezed data')
fig

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ppg_sample_complex)),
                         y= ppg_sample_complex,))
#fig.show(title='original data')
fig
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ppg_sample_squeezing)),
                         y= ppg_sample_squeezing))
#fig.show(title='squeezed data')
fig

In [ ]:
### Example of expanding function

In [ ]:
ecg_sample_expanding = signal.resample(ecg_sample_squeezing,
                                       int(len(ecg_sample_squeezing)*2))
ppg_sample_expanding = signal.resample(ppg_sample_squeezing,
                                       int(len(ppg_sample_squeezing)*2))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ppg_sample_complex)),
                         y= ppg_sample_complex,
                         name="original data"))
#fig.show(title='original data')
fig
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ppg_sample_expanding)),
                         y= ppg_sample_expanding,
                         name="the expanded data from the squeezed data"))
#fig.show()
fig

In [ ]:
# Example with bandpass filter

In [ ]:
from vital_sqi.common.band_filter import BandpassFilter

In [ ]:
butter_bandpass = BandpassFilter("butter",fs=256)
cheby_bandpass = BandpassFilter("cheby1",fs=256)
ellip_bandpass = BandpassFilter("ellip",fs=256)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y= ecg_sample_complex,
                         name='original signal'))
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y= butter_bandpass.signal_highpass_filter(
    ecg_sample_complex,cutoff=1,order=5),
                        name='highpass filtered - cutoff 1Hz'))
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y= butter_bandpass.signal_highpass_filter(
    ecg_sample_complex,cutoff=0.8,order=5),
                        name='highpass filtered - cutoff 0.8Hz'))
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y= butter_bandpass.signal_highpass_filter(
    ecg_sample_complex,cutoff=0.6,order=5),
                        name='highpass filtered - cutoff 0.6Hz'))
#fig.show()
fig

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ppg_sample_complex)),
                         y= ppg_sample_complex,
                         name='original signal'))
fig.add_trace(go.Scatter(x=np.arange(len(ppg_sample_complex)),
                         y= butter_bandpass.signal_highpass_filter(
    ppg_sample_complex,cutoff=1,order=5),
                        name='highpass filtered - cutoff 1Hz'))
fig.add_trace(go.Scatter(x=np.arange(len(ppg_sample_complex)),
                         y= butter_bandpass.signal_highpass_filter(
    ppg_sample_complex,cutoff=0.8,order=5),
                        name='highpass filtered - cutoff 0.8Hz'))
fig.add_trace(go.Scatter(x=np.arange(len(ppg_sample_complex)),
                         y= butter_bandpass.signal_highpass_filter(
    ppg_sample_complex,cutoff=0.6,order=5),
                        name='highpass filtered - cutoff 0.6Hz'))
#fig.show()
fig

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y= ecg_sample_complex,name='original signal'))
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y= butter_bandpass.signal_highpass_filter(
    ecg_sample_complex,cutoff=1,order=5)
                         ,name='butterworth highpass'))
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y= cheby_bandpass.signal_highpass_filter(
    ecg_sample_complex,cutoff=1,order=5)
                        ,name='chebyshev highpass'))
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y= ellip_bandpass.signal_highpass_filter(
    ecg_sample_complex,cutoff=1,order=5)
                        ,name='elliptic highpass'))
#fig.show()
fig

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(ppg_sample_complex)),
                         y= ppg_sample_complex,name='original signal'))
fig.add_trace(go.Scatter(x=np.arange(len(ppg_sample_complex)),
                         y= butter_bandpass.signal_highpass_filter(
    ppg_sample_complex,cutoff=1,order=5)
                         ,name='butterworth highpass'))
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y= cheby_bandpass.signal_highpass_filter(
    ppg_sample_complex,cutoff=1,order=5)
                        ,name='chebyshev highpass'))
fig.add_trace(go.Scatter(x=np.arange(len(ecg_sample_complex)),
                         y= ellip_bandpass.signal_highpass_filter(
    ppg_sample_complex,cutoff=1,order=5)
                        ,name='elliptic highpass'))
#fig.show()
fig